In [1]:
import os
import warnings
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
warnings.filterwarnings('ignore')

os.environ["TAVILY_API_KEY"]=TAVILY_API_KEY
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY

In [4]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

class State(TypedDict):
    messages: Annotated[list, add_messages]

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode, tools_condition

tool = TavilySearchResults(max_results=2)
tools=[tool]
tool_node = ToolNode(tools)

In [6]:
import json
from langchain_core.messages import ToolMessage
class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage"""
    def __init__(self, tools:list) -> None:
        self.tools_by_name={tool.name: tool for tool in tools}
    
    def __call__(self, inputs:dict):
        if messages := inputs.get("messages",[]):
            message = messages[-1]
        else:
            raise ValueError("no message found in input")
        outputs=[]
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"]
                )
            )
        return {"messages": outputs}

In [7]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o-mini")
llm_with_toold = llm.bind_tools(tools)

def chatbot(state: State):
    result=llm_with_toold.invoke(state["messages"])
    return {"messages": [result]}

In [9]:
from langgraph.graph import StateGraph
graph_builder=StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")
graph=graph_builder.compile(checkpointer=memory)

In [10]:
config={"configurable": {"thread_id": "1"}}
while True:
    user_input=input("User : ")
    if user_input.lower() in ["quit","q","exit"]:
        print("GoodBye")
        break
    for event in graph.stream({"messages": ("user", user_input)}, config):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

Assistant: 
Assistant: [{"url": "https://namu.wiki/w/대한민국+대통령", "content": "대한민국 대통령은 대한민국의 대통령 기관(機關)·직위(職位) 및 이를 수행하는 인물을 가리킨다. 대한민국의 현직 대통령은 제20대 윤석열이며, 임기는 2022년 5월 10일부터 2027년 5월 9일까지이다. 다만 2024년 12월 14일 부터는 같은 날에 열린 윤석열 대통령 탄핵소추안이 가결되어 윤석열 대통령의"}, {"url": "https://ko.wikipedia.org/wiki/대한민국의_대통령_목록", "content": "개인 도구 대한민국의 대통령 목록 도구 위키백과, 우리 모두의 백과사전. 제9차 개헌 이후의 현행 헌법에 따라 대통령의 임기는 5년이며 중임할 수 없지만,[1] 제9차 개헌 이전의 여러 헌법에서는 대통령의 임기가 고정되지 않았으며, 그로 인해 대통령직을 수행한 개인과 대통령 임기의 대수가 불일치하게 되었다. (생몰년)   정당  대[b]    임기  선거 / 득표율[5] ↑ 가 나 2016년 헌법에 위배되는 박근혜-최순실 게이트, 비선실세 의혹, 대기업 뇌물 의혹 등을 사유로 박근혜에 대한 탄핵소추안이 국회에서 가결되었고, 대통령 권한이 정지되어 황교안 국무총리가 권한대행을 맡았다.[27] 결과적으로 2017년 3월 10일 박근혜는 헌법재판소에 의해 전원일치로 대통령직에서 파면되었다.[28][29] ↑ 허정 (1979). ↑ KBS 뉴스 (2004). ↑ KBS 뉴스 (2017). ↑ 중앙일보 후보자 정보 (윤석열). 14 김영삼15 김대중16 노무현17 이명박18 박근혜19 문재인20 윤석열 대한민국의 대통령과 3부요인 및 헌법기관장 분류: 알찬 목록 후보 문서대한민국의 대통령대한민국 정치에 관한 목록대통령 목록대한민국 사람 목록 위키백과 소개"}]
Assistant: 현재 대한민국의 대통령은 윤석열이며, 그의 임기는 2022년 5월 10일부터 2027년 5월 9일까지입니다. 더 자세한 정보는 [

In [11]:
snapshot = graph.get_state(config)
snapshot

StateSnapshot(values={'messages': [HumanMessage(content='대한민국의 대통령은 누구야?', additional_kwargs={}, response_metadata={}, id='65a3e63f-c8f4-4146-9263-4df1d94ec646'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vCKCl9fpq7kfK8QDZDFWWV1h', 'function': {'arguments': '{"query":"대한민국의 대통령"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 87, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8cee6112-f345-433b-9619-120c8c02ee13-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '대한민국의 대통령'}, 'id': 'call_vCKCl9fpq7kfK8QDZDFWWV1h', 'ty

In [12]:
config2={"configurable": {"thread_id": "2"}}
graph.invoke({"messages": [{"role": "user", "content": "내 첫 질문이 뭐였어?"}]}, config2)

{'messages': [HumanMessage(content='내 첫 질문이 뭐였어?', additional_kwargs={}, response_metadata={}, id='fd715116-b226-4383-8f44-4fc6aab013df'),
  AIMessage(content='죄송하지만, 이전 대화 내용을 기억할 수 없습니다. 새로운 질문이 있으시면 도와드리겠습니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 87, 'total_tokens': 113, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-1d666c9e-ddab-461c-9460-f8a9fbdc4a92-0', usage_metadata={'input_tokens': 87, 'output_tokens': 26, 'total_tokens': 113, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

In [15]:
def filter_messages(messages: list):
    return messages[-2:]

def chatbot(state: State):
    messages=filter_messages(state["messages"])
    result=llm_with_toold.invoke(messages)
    return {"messages": [result]}
graph_builder=StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")
graph=graph_builder.compile(checkpointer=memory)

In [19]:
from langchain_core.messages import HumanMessage
config={"configurable": {"thread_id": "20"}}
input_message = HumanMessage(content="Hi! I'm bob and I like soccer")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

input_message = HumanMessage(content="What's my name?")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()


input_message = HumanMessage(content="What's my favorite?")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Hi! I'm bob and I like soccer
================================== Ai Message ==================================

Hi Bob! It's great to meet a soccer fan. Do you have a favorite team or player? Or is there a particular league or tournament you enjoy watching?
================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Bob. How can I assist you today?
================================ Human Message =================================

What's my favorite?
================================== Ai Message ==================================

I don't have information about your preferences or favorites. If you share what you're interested in, I can help you with that!
